## Viscosity of Alkanes

Description here "In this session..."

### Build the system

Brief description of what we'll do.

#### Import mBuild

In [ ]:
import mbuild as mb

#### Create class for a CH2 moiety

Here, we load the structure for a CH2 moeity from a PDB file and add two ports for the two dangling bonds on the carbon.

In [ ]:
class CH2(mb.Compound):
    def __init__(self):
        mb.load('utils/ch2.pdb', compound=self)
        self.add(mb.Port(anchor=self[0], orientation=[0, 1, 0], separation=0.07), 'up')
        self.add(mb.Port(anchor=self[0], orientation=[0, -1, 0], separation=0.07), 'down')

#### Create class for a hydrogen atom

Here, we create a hydrogen atom with a `Port` representing a single dangling bond.

In [ ]:
class H(mb.Compound):
    def __init__(self):
        self.add(mb.Compound(name='H'))
        self.add(mb.Port(anchor=self[0], orientation=[0, 1, 0], separation=0.07), 'up')

#### Create class for an alkane chain

Here, we define a class that stitches together CH2 moieties into an alkane chain of a user-defined length (hydrogen atoms are used to cap the ends of the chain to provide the complete chemistry).

In [ ]:
class Alkane(mb.Compound):
    """An alkane chain of a user-defined length."""
    def __init__(self, chain_length):
        """Initialize an Alkane Compound.

        Parameters
        ----------
        chain_length : int
            Length of the alkane chain (in number of carbons)
        """
        if n < 1:
            raise ValueError('n must be greater than 1')
        super(Alkane, self).__init__()

        chain = mb.Polymer(CH2(), n=n, port_labels=('up', 'down'))
        self.add(chain, 'chain')
        
        self.add(H(), 'up_cap')
        mb.force_overlap(move_this=self['up_cap'],
                         from_positions=self['up_cap']['up'],
                         to_positions=self['chain']['up'])
        
        self.add(H(), 'down_cap')
        mb.force_overlap(move_this=self['down_cap'],
                         from_positions=self['down_cap']['up'],
                         to_positions=self['chain']['down'])

#### Create class for a box of alkane chains

Description

In [ ]:
class AlkaneBox(mb.Compound):
    # Need to add an argument for initial box size/density
    def __init__(self, chain_length):
        # Need to fill with actual commands
        pass

### Choose a molecule

Using the class we defined above for creating a box of alkanes, instantiate a box of alkanes using a length of your choosing between 5 and 10 carbons.

e.g. `alkane_box = AlkaneBox(chain_length=8)`

In [ ]:
alkane_box = AlkaneBox(chain_length=)

#### Save to LAMMPS format

For saving we will need to provide a force field for Foyer to apply. For this simulation, we will use the Optimized Potentials for Liquid Simulations (OPLS) force field for all-atom systems.

In [ ]:
alkane_box.save('alkanes.lammps', forcefield_name='oplsaa', overwrite=True)

#### Perform the simulation!

First, however, lets take a look at the simulation script.

In [ ]:
!cat in.sllod